## MySQL에서 데이터 로드 

In [ ]:
#pip install pymysql

In [1]:
import pymysql
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
# MySQL 연결 정보
DB_HOST = "15.168.145.74" 
DB_USER = "lab08"  
DB_PASSWORD = "Multi1234!" 
DB_NAME = "my_db" 
#TABLE_NAME = "merge_car_siren"

In [3]:
# MySQL에서 데이터 가져오기
conn = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWORD, database=DB_NAME)
query = f"SELECT * FROM merge_car_siren"
query1 = f"SELECT * FROM merge_car_horn"

car_horn_df = pd.read_sql(query, conn) 
car_siren_df =  pd.read_sql(query1, conn)
conn.close()

/tmp/ipykernel_2554/2131438081.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  car_horn_df = pd.read_sql(query, conn)
/tmp/ipykernel_2554/2131438081.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  car_siren_df =  pd.read_sql(query1, conn)


## EDA

In [4]:
print(f'horn 데이터 개수: {car_horn_df.shape[0]}')
print(f'siren 데이터 개수: {car_siren_df.shape[0]}')

horn 데이터 개수: 1495
siren 데이터 개수: 1937


In [5]:
car_horn_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1495 entries, 0 to 1494
Data columns (total 46 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   fileName_wav   1495 non-null   object 
 1   fileName_json  1495 non-null   object 
 2   mfcc_1_wav     1495 non-null   float64
 3   mfcc_2_wav     1495 non-null   float64
 4   mfcc_3_wav     1495 non-null   float64
 5   mfcc_4_wav     1495 non-null   float64
 6   mfcc_5_wav     1495 non-null   float64
 7   mfcc_6_wav     1495 non-null   float64
 8   mfcc_7_wav     1495 non-null   float64
 9   mfcc_8_wav     1495 non-null   float64
 10  mfcc_9_wav     1495 non-null   float64
 11  mfcc_10_wav    1495 non-null   float64
 12  mfcc_11_wav    1495 non-null   float64
 13  mfcc_12_wav    1495 non-null   float64
 14  mfcc_13_wav    1495 non-null   float64
 15  area_start     1495 non-null   float64
 16  area_end       1495 non-null   float64
 17  category_01    1495 non-null   object 
 18  category

In [6]:
car_horn_df.iloc[0:5,15:]

,area_start,area_end,category_01,category_02,category_03,decibel,labelName,soundQuality,subCategory,bitRate,...,distance,district,latitude,longitude,micClass,obstacle,place,recordingTime,urban,weather
0,2.0,33.057,교통소음,자동차,차량사이렌,72,1.자동차_339_1.wav,노이즈,구급차,1411kbps,...,100m,공덕동,37.55,126.96,무지향성,없음,도로변,13:50,서울특별시,맑음
1,2.0,32.678,교통소음,자동차,차량사이렌,118,1.자동차_490_1.wav,노이즈,구급차,1411kbps,...,20m,한남동,37.53,127.01,무지향성,없음,도로변,13:00,서울특별시,맑음
2,2.0,32.310,교통소음,자동차,차량사이렌,100,1.자동차_353_1.wav,노이즈,구급차,1411kbps,...,10m,학동,35.14,126.92,무지향성,없음,도로변,12:15,광주광역시,맑음
3,2.0,32.197,교통소음,자동차,차량사이렌,88,1.자동차_546_1.wav,노이즈,구급차,1411kbps,...,12m,신영동,37.61,126.96,무지향성,없음,도로변,09:07,서울특별시,맑음
4,2.0,61.942,교통소음,자동차,차량사이렌,68,1.자동차_70695_1.wav,노이즈,구급차,705kbps,...,9m,중산동,37.50,126.57,무지향성,없음,도로변,01:51,인천광역시,맑음


In [7]:
#데이터프레임 병합 
df_combined = pd.concat([car_horn_df, car_siren_df], ignore_index=True)

df_combined.head()

,fileName_wav,fileName_json,mfcc_1_wav,mfcc_2_wav,mfcc_3_wav,mfcc_4_wav,mfcc_5_wav,mfcc_6_wav,mfcc_7_wav,mfcc_8_wav,...,distance,district,latitude,longitude,micClass,obstacle,place,recordingTime,urban,weather
0,1.car_siren_339.wav,1.car_siren_339.wav,-359.73535,182.16605,-23.895273,22.655750,-10.495936,11.991384,6.407280,1.399655,...,100m,공덕동,37.55,126.96,무지향성,없음,도로변,13:50,서울특별시,맑음
1,1.car_siren_490.wav,1.car_siren_490.wav,-191.29960,212.07877,0.141339,9.329338,-4.937894,10.529773,1.791591,2.207042,...,20m,한남동,37.53,127.01,무지향성,없음,도로변,13:00,서울특별시,맑음
2,1.car_siren_353.wav,1.car_siren_353.wav,-281.63776,218.64851,-7.122757,-15.105639,14.932013,8.161676,10.723100,0.602772,...,10m,학동,35.14,126.92,무지향성,없음,도로변,12:15,광주광역시,맑음
3,1.car_siren_546.wav,1.car_siren_546.wav,-304.95110,225.62074,-31.841942,4.019028,-6.455601,-8.875257,7.754140,-0.778751,...,12m,신영동,37.61,126.96,무지향성,없음,도로변,09:07,서울특별시,맑음
4,1.car_siren_70695.wav,1.car_siren_70695.wav,-563.17760,142.45625,54.817646,2.027122,-13.447221,-0.668920,16.466820,8.719242,...,9m,중산동,37.50,126.57,무지향성,없음,도로변,01:51,인천광역시,맑음


In [8]:
df_combined.iloc[:, 2:15]

,mfcc_1_wav,mfcc_2_wav,mfcc_3_wav,mfcc_4_wav,mfcc_5_wav,mfcc_6_wav,mfcc_7_wav,mfcc_8_wav,mfcc_9_wav,mfcc_10_wav,mfcc_11_wav,mfcc_12_wav,mfcc_13_wav
0,-359.73535,182.16605,-23.895273,22.655750,-10.495936,11.991384,6.407280,1.399655,14.864681,-1.322799,11.512208,-3.289162,3.086125
1,-191.29960,212.07877,0.141339,9.329338,-4.937894,10.529773,1.791591,2.207042,8.355843,11.349453,2.366396,0.389569,6.672070
2,-281.63776,218.64851,-7.122757,-15.105639,14.932013,8.161676,10.723100,0.602772,-6.376552,6.390977,10.868795,3.860860,-1.086853
3,-304.95110,225.62074,-31.841942,4.019028,-6.455601,-8.875257,7.754140,-0.778751,14.105545,9.251060,2.774182,8.414690,6.328545
4,-563.17760,142.45625,54.817646,2.027122,-13.447221,-0.668920,16.466820,8.719242,6.064907,5.030265,9.305651,19.866758,24.583746
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3427,-296.59660,201.82323,-41.995213,38.293407,15.336274,8.951137,2.428992,0.401730,23.885618,0.368696,0.427634,13.866289,0.759625
3428,-264.83923,197.21075,-77.277880,23.032236,11.380154,7.896886,11.269576,-4.560492,17.224950,4.160231,-1.057662,0.926013,-7.930759
3429,-228.09323,64.34398,-13.663012,41.246460,-7.827433,21.293640,-13.415158,3.358162,5.326135,-3.165687,-11.261456,-1.512664,6.411310
3430,-426.90427,130.72240,47.745327,8.306962,5.675391,15.449170,5.677291,6.208929,-0.195498,-5.397255,-2.674880,-6.652485,-10.272819


## 모델 생성

In [17]:
#import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # warning 메시지 숨기기

import tensorflow as tf
print("Is GPU available:", tf.test.is_gpu_available())

import tensorflow as tf
print('tf.__version__:',tf.__version__)

Is GPU available: True
tf.__version__: 2.13.0


2025-02-20 09:31:02.969323: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-20 09:31:02.971102: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-20 09:31:02.972572: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [10]:
tf.config.set_visible_devices([], 'GPU')

2025-02-20 09:25:52.619788: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-20 09:25:52.621343: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-20 09:25:52.622752: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [11]:
# 레이블(label)과 피처(features) 분리
X = df_combined.iloc[:, 2:15] # 원하는 MFCC 열만 선택
y = df_combined['category_03']  #차량경적, 차량사이렌

In [12]:
# 범주형 데이터(레이블) 숫자로 변환
y = y.astype('category').cat.codes

In [13]:
# 데이터셋 분할 (train: 80%, test: 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# X_train을 numpy 배열로 변환 후 reshape
X_train = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

# 클래스 수 (필요에 따라 수정)
num_classes = len(set(y))

model = Sequential()
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
history = model.fit(X_train, y_train, epochs=50, batch_size=1, validation_data=(X_test, y_test))

Epoch 1/50


2025-02-20 09:25:58.481268: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fd324008d90 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2025-02-20 09:25:58.481321: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2025-02-20 09:25:58.566464: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


  87/2745 [..............................] - ETA: 4s - loss: 1.3473 - accuracy: 0.7816   

2025-02-20 09:25:58.804219: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2745/2745 [==============================] - 7s 2ms/step - loss: 0.4255 - accuracy: 0.9304 - val_loss: 0.0527 - val_accuracy: 0.9811
Epoch 2/50
2745/2745 [==============================] - 6s 2ms/step - loss: 0.1016 - accuracy: 0.9639 - val_loss: 0.0730 - val_accuracy: 0.9651
Epoch 3/50
2745/2745 [==============================] - 5s 2ms/step - loss: 0.0972 - accuracy: 0.9709 - val_loss: 0.0883 - val_accuracy: 0.9563
Epoch 4/50
2745/2745 [==============================] - 5s 2ms/step - loss: 0.0762 - accuracy: 0.9719 - val_loss: 0.0535 - val_accuracy: 0.9782
Epoch 5/50
2745/2745 [==============================] - 5s 2ms/step - loss: 0.0590 - accuracy: 0.9821 - val_loss: 0.0403 - val_accuracy: 0.9825
Epoch 6/50
2745/2745 [==============================] - 5s 2ms/step - loss: 0.0702 - accuracy: 0.9756 - val_loss: 0.0826 - val_accuracy: 0.9709
Epoch 7/50
2745/2745 [==============================] - 5s 2ms/step - loss: 0.0499 - accuracy: 0.9800 - val_loss: 0.1782 - val_accuracy: 0.9447
Epo